In [ ]:
!pip install plotly
!pip install lazypredict

In [ ]:
import glob
import pandas as pd

csv_files = glob.glob("*d.csv")
#csv_files = glob.glob("*w.csv")
print(csv_files)

dataset = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)
dataset.to_csv("dataset_normal.csv", index=False)

['50_wled.csv', '2k_wled.csv', '200_wled.csv', '10_wled.csv', '500_wled.csv', '20_wled.csv', '100_wled.csv', 'blank_wled.csv']


In [ ]:
import pandas as pd
from lazypredict.Supervised import LazyClassifier
import plotly.express as px
#from pandas_profiling import ProfileReport

In [ ]:
df = pd.read_csv("/content/dataset_normal.csv")
# df_raw = pd.read_csv("/content/dataset_tcs_inr_raw.csv")
# uv = pd.read_csv("/content/dataset_tcs_inr_uv.csv")
# uv_raw = pd.read_csv("/content/dataset_tcs_inr_uv_raw.csv")

In [ ]:
df.head()
df.tail()
df.describe()
df.info()
df.columns
df.dtypes

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1334 entries, 0 to 1654
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   R1      1334 non-null   int64 
 1   G1      1334 non-null   int64 
 2   B1      1334 non-null   int64 
 3   R2      1334 non-null   int64 
 4   G2      1334 non-null   int64 
 5   B2      1334 non-null   int64 
 6   class   1334 non-null   object
dtypes: int64(6), object(1)
memory usage: 83.4+ KB


R1        int64
G1        int64
B1        int64
R2        int64
G2        int64
B2        int64
class    object
dtype: object

# **Separate X and y**

In [ ]:
features = df.drop(['class'], axis=1)
labels = df['class']
classes = labels.unique()
columns = features.columns.tolist()
print(classes)
print(columns)

['50' '2k' '200' '10' '500' '20' '100' 'blank']
['R1', 'G1', 'B1', 'R2', 'G2', 'B2']


In [ ]:

# Find outliers
print('Outliers')
for cls in classes:
  df_cls = df[df['class'] == cls]
  des_cls = df_cls.describe()
  print(f'\nFor {cls}')
  print(des_cls, '\n')
  for column in columns:
    q3 = des_cls[column][-2]
    q1 = des_cls[column][-4]
    iqr = q3 - q1
    up = q3 + (2.5 * iqr)
    lw = q3 - (2.5 * iqr)
    outliers = df_cls[(df_cls[column] < lw) | (df_cls[column] > up)]
    # print(f'outliers =  {outliers} \n')
    print(f'{column}: {len(outliers)}')
    print(f'up: {up}, lw: {lw}')



Outliers

For 50
          R1     G1     B1     R2     G2     B2
count 201.00 201.00 201.00 201.00 201.00 201.00
mean   31.21 112.34 110.00  67.10 158.43 128.61
std    15.01  54.60  58.69  38.04  70.62  69.48
min     4.00  16.00  16.00   8.00  22.00  16.00
25%    22.00  75.00  69.00  38.00  93.00  68.00
50%    28.00  99.00  99.00  61.00 162.00 117.00
75%    37.00 149.00 149.00  83.00 219.00 180.00
max    96.00 255.00 255.00 228.00 255.00 255.00 

R1: 4
up: 74.5, lw: -0.5
G1: 0
up: 334.0, lw: -36.0
B1: 0
up: 349.0, lw: -51.0
R2: 2
up: 195.5, lw: -29.5
G2: 0
up: 534.0, lw: -96.0
B2: 0
up: 460.0, lw: -100.0

For 2k
          R1     G1     B1     R2     G2     B2
count 202.00 202.00 202.00 202.00 202.00 202.00
mean   55.12  47.85  74.99  70.52 142.07 123.42
std    35.68  33.88  50.34  39.92  72.15  64.78
min     6.00  11.00  17.00   8.00  21.00  21.00
25%    34.00  26.00  45.00  40.00  82.00  69.00
50%    43.00  34.00  56.50  62.50 127.00 105.00
75%    65.00  59.00  88.75  98.00 216.00 186

In [ ]:
df["class"].unique()

array(['50', '2k', '200', '10', '500', '20', '100', 'blank'], dtype=object)

In [ ]:
# Remove outliers
df.drop(df[(df['class'] == '100') & ((df['R1'] < 12) | (df['G1'] > 90) | (df['G1'] < 12) | (df['B1'] < 12)| (df['B1'] > 130))].index, inplace=True)
df.drop(df[(df['class'] == '20') & ((df['R1'] < 20) | (df['R1'] > 125) | (df['G1'] < 33.0) | (df['G1'] > 150) | (df['B1'] < 16.5)|(df['B1'] > 55))].index, inplace=True)
df.drop(df[(df['class'] == '200') & ((df['R1'] < 20) | (df['R1'] >220) | (df['G1'] < 18) | (df['G1'] >140) | (df['B1'] < 15.5))].index, inplace=True)
df.drop(df[(df['class'] == '2k') & ((df['R1'] > 120) |(df['R1'] < 20) | (df['G1'] > 80) | (df['B1'] >160) )].index, inplace=True)
df.drop(df[(df['class'] == '50') & ((df['R1'] > 70) | (df['B1'] < 25.5))].index, inplace=True)
df.drop(df[(df['class'] == '10') & ((df['R1'] < 35) | (df['R1'] > 80) | (df['G1'] < 12) | (df['G1'] >80 ) | (df['B1'] < 12))].index, inplace=True)
df.drop(df[(df['class'] == '500') & ((df['R1'] < 12))].index, inplace=True)
# df.drop(df[(df['class'] == '100') & (df['G1'] > 18.5)].index, inplace=True)
# df.drop(df[(df['class'] == '200') & ((df['R1'] < 25) | (df['G1'] > 19.5))].index, inplace=True)


In [ ]:
df["class"].unique()

array(['50', '2k', '200', '10', '500', '20', '100', 'blank'], dtype=object)

In [ ]:
# df[(df['class'] == '20') & (df['B1'] > 4)].index

In [ ]:
# df.to_csv('dataset_normal.csv', index=False)

In [ ]:
fig = px.scatter_3d(df, x='R1', y='G1', z='B1',
              color='class',animation_group= "class")
fig.update_traces(marker_size=4)
fig.show()

In [ ]:
fig = px.scatter_3d(df, x='R2', y='G2', z='B2',
              color='class',animation_group= "class")
fig.update_traces(marker_size=4)
fig.show()

In [ ]:
df["class"].unique()

array(['50', '2k', '200', '10', '500', '20', '100', 'blank'], dtype=object)

In [ ]:
df = df[(df['class'] == '500') | (df['class'] == '2k') | (df['class'] == '200') | (df['class'] == '10') | (df['class'] == '20') | (df['class'] == '100') | (df['class'] == '50') | (df['class'] == 'blank')]

In [ ]:
df['class'].value_counts()

blank    234
500      192
50       189
20       172
2k       163
100      160
200      143
10        81
Name: class, dtype: int64

In [ ]:
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn import model_selection, tree, metrics, ensemble
import numpy as np
import matplotlib.pyplot as plt

In [ ]:

cols = ['R1', 'G1', 'B1'] #,'R2','G2','B2'
features = df.drop(columns=['class'])[cols]
label = df['class']
X, x, Y, y = model_selection.train_test_split(features, label, stratify=label, test_size=0.2, random_state=42)

In [ ]:
features


,R1,G1,B1
0,45,127,106
1,45,127,106
2,46,130,108
3,35,93,78
4,19,49,42
...,...,...,...
1650,6,16,18
1651,7,16,18
1652,7,16,18
1653,7,16,18


In [ ]:
label.value_counts()

blank    234
500      192
50       189
20       172
2k       163
100      160
200      143
10        81
Name: class, dtype: int64

In [ ]:
clf = LazyClassifier()

predictions, scores = clf.fit(X, x, Y, y)

 86%|████████▌ | 25/29 [00:01<00:00, 16.48it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000699 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 286
[LightGBM] [Info] Number of data points in the train set: 1067, number of used features: 3
[LightGBM] [Info] Start training from score -2.798219
[LightGBM] [Info] Start training from score -2.120576
[LightGBM] [Info] Start training from score -2.045353
[LightGBM] [Info] Start training from score -2.236408
[LightGBM] [Info] Start training from score -2.105072
[LightGBM] [Info] Start training from score -1.955326
[LightGBM] [Info] Start training from score -1.935654
[LightGBM] [Info] Start training from score -1.741498
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

100%|██████████| 29/29 [00:02<00:00, 11.44it/s]


In [ ]:
predictions

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
SVC,1.00,1.00,None,1.00,0.06
RandomForestClassifier,1.00,1.00,None,1.00,0.22
ExtraTreesClassifier,1.00,1.00,None,1.00,0.18
QuadraticDiscriminantAnalysis,1.00,1.00,None,1.00,0.04
KNeighborsClassifier,1.00,1.00,None,1.00,0.03
BaggingClassifier,0.99,0.99,None,0.99,0.05
LabelPropagation,0.99,0.99,None,0.99,0.09
LabelSpreading,0.99,0.99,None,0.99,0.11
DecisionTreeClassifier,0.99,0.99,None,0.99,0.02


# <h1>**training**

In [ ]:

cols = ['R1', 'G1', 'B1'] #,'R2','G2','B2'
features = df.drop(columns=['class'])[cols]
label = df['class']
X, x, Y, y = model_selection.train_test_split(features, label, stratify=label, test_size=0.2, random_state=42)

In [ ]:
classes = df["class"].unique().tolist()
classes.sort()
print(classes)   # Create classes list in pico program

['10', '100', '20', '200', '2k', '50', '500', 'blank']


In [ ]:
features

,R1,G1,B1
0,45,127,106
1,45,127,106
2,46,130,108
3,35,93,78
4,19,49,42
...,...,...,...
1650,6,16,18
1651,7,16,18
1652,7,16,18
1653,7,16,18


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Create a decision tree classifier
classifier = DecisionTreeClassifier()

# Train the classifier on the training data
classifier.fit(X, Y)

# Make predictions on the test data
y_pred = classifier.predict(x)


In [ ]:
x.describe()

,R1,G1,B1
count,267.00,267.00,267.00
mean,43.62,56.00,49.21
std,37.19,37.87,38.98
min,6.00,15.00,17.00
25%,20.00,29.00,21.50
50%,34.00,50.00,37.00
75%,57.00,70.00,64.50
max,210.00,255.00,255.00


In [ ]:
# Evaluate the classifier
accuracy = accuracy_score(y, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9887640449438202


In [ ]:
y_pred = classifier.predict([[6,16,18]])
print(y_pred)

['blank']


In [ ]:

print(
    f"Classification report for classifier {classifier}:\n"
    f"{metrics.classification_report(y, y_pred)}\n"
)

ValueError: ignored

In [ ]:
!pip install m2cgen
!pip install python-minimizer

In [ ]:
import sys
import m2cgen as m2c

sys.setrecursionlimit(999999999)
# Convert model to python code
code = m2c.export_to_python(classifier, function_name="classify")
with open("classifier_normal_dtc.py", "w") as fp:
  fp.write(code)

In [ ]:
code

In [ ]:
!python-minimizer classifier_normal_dtc.py -o classifier_norm_dtc.py

In [ ]:
from classifier_norm_dtc import classify

In [ ]:
pred = classify([9,16,18])

print(pred)

print(len(pred))

print(classes)

if pred is not None:
  pred_idx = pred.index(max(pred))
  pred_d = classes[pred_idx]
  print(pred_d)

using linear **scv**

In [ ]:
cols = ['R1', 'G1', 'B1'] #,'R2','G2','B2'
features = df.drop(columns=['class'])[cols]
label = df['class']
X, x, Y, y = model_selection.train_test_split(features, label, stratify=label, test_size=0.2, random_state=42)

In [ ]:
from sklearn.svm import LinearSVC
classifier = LinearSVC()
classifier.fit(X, Y)

In [ ]:
pred= classifier.predict(x)
accuracy = accuracy_score(y,pred)
print("Accuracy:", accuracy)

In [ ]:
print(
    f"Classification report for classifier {classifier}:\n"
    f"{metrics.classification_report(y, pred)}\n"
)

In [ ]:
import sys
import m2cgen as m2c

sys.setrecursionlimit(999999999)
# Convert model to python code
code = m2c.export_to_python(classifier, function_name="classify")
with open("classifier_normal_svc1.py", "w") as fp:
  fp.write(code)

In [ ]:
code


In [ ]:
!python-minimizer classifier_normal_svc1.py -o classifier_norm_svc1.py

In [ ]:
from classifier_norm_svc1 import classify

In [ ]:
pred = classify([65,92,120])

print(pred)

print(pred.index(max(pred)))

print(classes)

**Traning** **RF**

In [ ]:
# Import the necessary libraries

from sklearn.ensemble import RandomForestClassifier



# Create the random forest model
model = RandomForestClassifier(n_estimators=100)

# Train the model
model.fit(X, Y)

# Evaluate the model
score = model.score(x, y)
print('The accuracy of the model is:', score)

In [ ]:
import sys
import m2cgen as m2c

sys.setrecursionlimit(999999999)
# Convert model to python code
code = m2c.export_to_python(classifier, function_name="classify")
with open("classifier_normal_rf.py", "w") as fp:
  fp.write(code)

In [ ]:
code

In [ ]:
!python-minimizer classifier_normal_rf.py -o classifier_norm_rf.py

In [ ]:
from classifier_norm_svc1 import classify

In [ ]:
pred = classify([7,17,16])

print(pred)

print(pred.index(max(pred)))

print(classes)